In [2]:
library(Seurat)
library(ggplot2)
library(patchwork)
library(Matrix)

Attaching SeuratObject



### Read in rep1 avr

In [3]:
data_folder <- "../../data"

In [4]:
seq_folder = file.path(data_folder, 'AvrRpt2_alone2.rds')

In [5]:
scrna = readRDS(seq_folder)

In [6]:
rep1_folder = file.path(data_folder, 'AvrRpt2_alone.rds')
rep1 = readRDS(rep1_folder)
rep1_pseudotime = read.csv(file.path(data_folder, 'temp_objects', 'pseudotime_csv.csv'))
rep1@meta.data$index <- rownames(rep1@meta.data)
cleaned_vector <- gsub("col_|_rep1", "", rownames(scrna@meta.data))
copyscrna <- scrna@meta.data
rownames(copyscrna) <- cleaned_vector

In [7]:
rep1@meta.data <- merge(rep1@meta.data, copyscrna[c("sub_clst_rna", 'celltype', 'SCT_snn_res.1')], by = 0, all.x = TRUE)

In [8]:
rownames(rep1@meta.data) <- rep1@meta.data$Row.names

In [9]:
Idents(object = rep1) <- "celltype.y"

In [10]:
rep1 <- subset(rep1, idents = c('Vasculature', 'Mesophyll', 'Epidermis', 'Unknown'))

Loading required package: Signac


Attaching package: 'Signac'


The following object is masked from 'package:Seurat':

    FoldChange


Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from pca_kt57_ to pcakt57_"


In [11]:
merged_df <- merge(rep1@meta.data, rep1_pseudotime, by = 'index', all.x = TRUE)
rep1@meta.data <- merged_df

In [12]:
rownames(rep1@meta.data) <- rep1@meta.data$index

In [13]:
DefaultAssay(rep1) <- "SCT"

In [14]:
rep1@meta.data$celltype <- ifelse(rep1@meta.data$celltype.y == "", "Unknown", rep1@meta.data$celltype.y)
rep1 <- subset(x = rep1, subset = celltype.y != c('Unknown'))

### read in dc3000

In [15]:
dc3000 <- readRDS(file.path(data_folder, 'DC3000_alone.rds'))

In [16]:
dc3000 <- subset(dc3000, subset = sample.order == '01_DC3000_24h')


Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from pca_kt56_ to pcakt56_"


In [17]:
anchors <- list()

In [18]:
anchors[[1]] <- FindTransferAnchors(
    reference = rep1,
    query = dc3000,
    reference.reduction = "pca", 
    normalization.method = "SCT",
    dims = 1:50
)

Normalizing query using reference SCT model



Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 754 anchors



In [19]:

# Replace NaN values with 0 in the 'dpt_pseudotime' column
rep1@meta.data$dpt_pseudotime[is.na(rep1@meta.data$dpt_pseudotime)] <- 0

sample_vector <- rep1@meta.data$dpt_pseudotime 
sparse_matrix <- t(sparseMatrix(i = 1:length(sample_vector),
                              j = rep(1, length(sample_vector)),
                              x = sample_vector))
rep1[['pseudotime']] = t(sparse_matrix)
colnames(sparse_matrix) <- colnames(rep1)
rownames(sparse_matrix) <- c('pseudotime')

In [20]:
dc3000 <- MapQuery(
    anchorset = anchors[[1]], 
    query = dc3000,
    reference = rep1, 
    refdata = list(pseudotime = sparse_matrix),
    reference.reduction = "pca",
)

Finding integration vectors



Finding integration vector weights

Transfering 1 features onto reference data

Integrating dataset 2 with reference dataset

Finding integration vectors

Integrating data

Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from ref.pca_ to refpca_"
Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from ref.pca_ to refpca_"
Warning message:
"All keys should be one or more alphanumeric characters followed by an underscore '_', setting key to refpca_"


In [21]:
sparse_pseudo <- dc3000@assays$pseudotime@data
sparse_vector <- sparse_pseudo[1,]
dense_vector <- as.vector(sparse_vector)

In [22]:
dc3000@meta.data$pseudotime <- dense_vector 

In [23]:
dc3000@meta.data$pseudotime[is.na(dc3000@meta.data$pseudotime)] <- 0

In [24]:
dior::write_h5(dc3000, file= file.path(data_folder, "DC3000_alone.h5"), object.type = 'seurat')

Warning message in matrix_to_h5(mat = slot_layer@data, h5 = layers, gr_name = s):
"s is dense matrix"
